In [37]:
import pandas as pd
import numpy as np

housing = pd.read_csv('manhattan_rentals_apr27.csv')
neighborhoods = [x.strip() for x in list(housing['Neighborhood'].value_counts().index)]
neighborhood_options = [{'label': x, 'value': x} for x in neighborhoods]
full_df = pd.read_csv('scored_housing/full_housing.csv')
full_df['Monthly Rental Price'] = full_df['Monthly Rental Price'].astype(float)

import chart_studio.plotly as py
from plotly.offline import plot
import plotly.graph_objs as go
import plotly
import chart_studio as cs
from sklearn.metrics.pairwise import cosine_similarity

## Creating the User Intake Survey on Dash

In [149]:
dislikes = []
new_df = pd.DataFrame()

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


navbar2 = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Page 1", href="#")),
        dbc.DropdownMenu(
            children=[
                dbc.DropdownMenuItem("More pages", header=True),
                dbc.DropdownMenuItem("Page 2", href="#"),
                dbc.DropdownMenuItem("Page 3", href="#"),
            ],
            nav=True,
            in_navbar=True,
            label="More",
        ),
    ],
    brand="NavbarSimple",
    brand_href="#",
    color="primary",
    dark=True,
)
app.config['suppress_callback_exceptions']=True

navbar = html.Div(
    children=[
        dcc.Location(id='url', refresh=False),

    dcc.Link('Home', href='/'),
#     html.Br(),
#         html.Pre(),
        dcc.Markdown(children="""   """),
    dcc.Link('About Us', href='/about-us')
    ], style={
                'textAlign':'right',
               'background-color': 'skyblue',
      'overflow': 'hidden',
      'position': 'relative',
      'bottom': '0',
      'width': '50px'
           }
)

# add callback for toggling the collapse on small screens



footer = html.Footer([
    html.Div(children=["Made with ❤️ in Berkeley, CA."], style={
        
        'color': 'white',
        'font-size': '16px'
        
    })],style={
            'textAlign':'left',
            'background-color': '#333',
            'overflow': 'hidden',
            'position': 'relative',
            'bottom': '0',
            'width': '100%',
            'height': '25px',
            'padding': '10px',
            'margin-top': '20px'
           })


app.layout = html.Div(children=[
    
    navbar,
    
    
    html.Div(id='display'),
    
    footer


    ])


@app.callback(dash.dependencies.Output('display', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display(pathname):
    if pathname == '/about-us':
        return html.Div([  
            
            html.H1(children=["Welcome to ApartmentRanker v.2!"],
           style={
               'textAlign':'center'
           }),
        html.Div('You are on page {}'.format(pathname))
    ])
    
    if pathname == "/":
        return html.Div([html.Div(children=[
            
              html.H1(children=["Welcome to ApartmentRanker!"],
           style={
               'textAlign':'center'
           }),
            
        html.H3(children="Tell us about yourself!",
               style={
                   'textAlign':'center'
               }),
        
        html.Div([
            dcc.Markdown(children=
                        """
                        ApartmentRanker works best if you answer each question honestly as we use this data to match suitable apartments with you. Simply fill out the required questions and see apartments around you today!
                        """)
        ]),
        
        html.Div(children=[
            
            html.H5(children="First, let's talk requirements.",
                    style={'textAlign':'center'}),
            
            html.Div(children=[
                html.Label('Number of bedrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bedrooms-state',
                    options=[
                        {'label': '0 (studio)','value':0.0},
                        {'label': '1 bedroom','value':1.0},
                        {'label': '2 bedrooms','value':2.0},
                        {'label': '3 bedrooms','value':3.0},
                        {'label': '4 bedrooms','value':4.0},
                        {'label': '5 bedrooms','value':5.0}
                    ]),

                html.Label('Number of bathrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bathrooms-state',
                    options=[
                        {'label': '1 bathroom','value':1.0},
                        {'label': '2 bathrooms','value':2.0},
                        {'label': '3 bathrooms','value':3.0},
                        {'label': '4 bathrooms','value':4.0},
                        {'label': '5 bathrooms','value':5.0}
                    ]),

                ],style={'columnCount':2}),
        
            html.Label('Maximum monthly rental price',style={"font-weight":"bold"}),
            dcc.Slider(
                id='price_max-state',
                min=1000,
                max=10000,
                marks={i: '${}'.format(i) for i in range (1000,11000,1000)}
            ),
            
            html.Label('Preferred neighborhood',style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='neighborhood-state',
                options=neighborhood_options,
                multi=False)
        
        ]),
        
        html.Div(children=[
            
            html.H5(children="One last thing, tell us about your priorities!",
                    style={'textAlign':'center'}),
            
            html.Label("1st priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority1-state',
                value='grocer',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("2nd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority2-state',
                value='nightlife',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),    
                 
            html.Label("3rd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority3-state',
                value='noise',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),   
                 
            html.Label("4th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority4-state',
                value='food',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("5th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority5-state',
                value='subway',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),  
                 
            html.Label("6th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority6-state',
                value='trees',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),                 
            ],
                style=
                    {
                        'margin-top':'10px',
                        'margin-bottom':'10px'
                    }),
        
        html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
        
        # INITIAL RECOMMENDATION 
            
        html.Div(id='hide-me-success',children=[
            
            html.H4(children="Woot! 🎉", style={'textAlign':'left','margin-bottom':'-5px','font-weight':'bold'}),

            html.Div(children="We found a recommendation for you.",style={'textAlign':'left'}),
            
            html.Div(id='mappy-boi',
                    style={
                        'width':'325px',
                        'height':'325px',
                        'background':'#003262',
                        'float':'left',
                        'margin-right':'20px',
                        'margin-top':'10px'
                    }),
            
            html.Div(children=[
                html.H5(id='initial_recc_name', style={'margin-bottom':'-10px', 'font-weight':'bold'}),
                html.H6(id='initial_recc_neigh', style={'margin-bottom':'-2px', 'font-style':'oblique'}),
                html.Div(id='initial_recc_sent')],
                    style={
                        'textAlign':'left'
                    }),
            html.Div(children=[
                html.Li(id='grocer-num'),
                html.Li(id='nightlife-num'),
                html.Li(id='noise-num'),
                html.Li(id='resto-num'),
                html.Li(id='sub-num'),
                html.Li(id='tree-num')],
                    style={
                        'margin-top': '10px',
                        'margin-left':'50px',
                        'font-size':'12px',
                        'textAlign':'left'
                    }),
            
            html.Div(children=[
                html.Button(children="👍"),
                html.Button(children="👎")],
                    style={
                       "columnCount":2 
                    })
            ],
                style={
                    'display':'none',
                    'margin-bottom':'20px'
                }),

        html.Div(id='hide-me-failure', children=[
            
            html.H4(children="Hmm...🤔", style={'textAlign':'left','margin-bottom':'-5px','font-weight':'bold'}),
            html.Div(children="We couldn't find anything with this criteria.",style={'textAlign':'left'})],
                style={
                    'display':'none'
                })
        ],
    style=
        {"margin":"0 auto",
         'max-width':"650px",
         'textAlign':'center'
        }
    )])
    
@app.callback(
    [Output('initial_recc_name','children'),
     Output('initial_recc_neigh','children'),
     Output('initial_recc_sent','children'),
     Output('grocer-num','children'),
     Output('nightlife-num','children'),
     Output('noise-num','children'),
     Output('resto-num','children'),
     Output('sub-num','children'),
     Output('tree-num','children'),
     Output('hide-me-success','style'),
     Output('hide-me-failure','style')],
    [Input('submit-button-state','n_clicks')],
    [State('bedrooms-state','value'),
    State('bathrooms-state','value'), 
    State('price_max-state','value'),
    State('neighborhood-state','value'),
    State('priority1-state','value'),
    State('priority2-state','value'),
    State('priority3-state','value'),
    State('priority4-state','value'),
    State('priority5-state','value'),
    State('priority6-state','value')])
def initial_rec(n_clicks,beds,bath,price,neighbor,p1,p2,p3,p4,p5,p6):
    #preferences is a list with the ranked features of
    # grocers_normalized
    # nightlife_normalized
    # noise_normalized
    # restaurants_normalized
    # subways_normalized
    # trees_normalized

    if n_clicks is None:
        return ""
    
    counter=1

    tree_pref = 0
    subway_pref = 0
    grocer_pref = 0
    noise_pref = 0
    restaurants_pref = 0
    nightlife_pref = 0

    for p in [p1,p2,p3,p4,p5,p6]:
        if p=="trees":
            tree_pref=counter
            counter+=1
            continue
        if p=="subway":
            subway_pref=counter
            counter+=1
            continue
        if p=="grocer":
            grocer_pref=counter
            counter+=1
            continue
        if p=="noise":
            noise_pref=counter
            counter+=1
            continue
        if p=="food":
            restaurants_pref=counter
            counter+=1
            continue
        if p=="nightlife":
            nightlife_pref=counter
            counter+=1
            continue
            
    preferences = np.array([grocer_pref, nightlife_pref, noise_pref, restaurants_pref, subway_pref, tree_pref])
    
    new_df = full_df[(full_df['Bedrooms'] == beds) & (full_df['Bathrooms'] == bath) & (full_df['Monthly Rental Price'] <= price) & (full_df['Neighborhood'] == neighbor)]
    new_df["User Score"] = 0.0
    
    scores = []
    for i in range(new_df.shape[0]):
        score = preferences * np.array(new_df.iloc[i, 19:25] + 10)
        scores.append(sum(score))
    new_df['User Score'] = scores
    new_df = new_df.sort_values(by = ['User Score'], ascending = False)
    
    counter_1 = 0
#     print(new_df.head())
    try:
        while True:
            potential = new_df.iloc[counter_1, :]
            if potential not in dislikes:
                break
            else:
                counter_1 = counter_1 + 1
        result = potential

        #recc = "We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) + " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms."
        apt_name = result['Location']
        apt_neigh = result['Neighborhood']
        recc_details = "This listing is $" + str(int(result['Monthly Rental Price'])) + " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms."
        apt_grocers = str(result['num_grocers']) + " grocery stores nearby." 
        apt_nightlife = str(result['num_nightlife']) + " nightlife spots nearby."
        apt_noise = str(result['num_noise'])  + " noise complaints in the last three months."
        apt_restos = str(result['num_restaurants']) + " restaurants or cafes nearby."
        apt_transit = str(result['num_subways'])  + " subway stops in walking distance."
        apt_trees = str(result['num_trees']) + " trees on nearby streets."
        
        return apt_name, apt_neigh, recc_details, apt_grocers, apt_nightlife, apt_noise, apt_restos, apt_transit, apt_trees, {'display':'block'}, {'display':'none'}
    except:
        return "None", "None", "None", "None", "None", "None", "None", "None", "There is currently no listing with the given criteria.", {'display':'none'}, {'display':'block'}
    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


## TESTING GROUNDS TO IMPLEMENT ANDERSON'S CODE

In [26]:
dislikes = []
prev_recc = 0

In [ ]:
dislikes = []
prev_recc = 0
first_recommendation = 0

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

cs.tools.set_credentials_file(username='apartmentranker', api_key='pk.eyJ1IjoiYXBhcnRtZW50cmFua2VyIiwiYSI6ImNrOWk0MGphdTAzY3QzZXA1NXZkNzB3MjIifQ.633TMyX6NjcaiooIn6204g')
mapbox_access_token = 'pk.eyJ1IjoiYXBhcnRtZW50cmFua2VyIiwiYSI6ImNrOWk0MGphdTAzY3QzZXA1NXZkNzB3MjIifQ.633TMyX6NjcaiooIn6204g'

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.config['suppress_callback_exceptions']=True

navbar = html.Header([
            html.Div(dcc.Location(id='url',refresh=False)),
            html.Div(children=["ApartmentRanker"], 
                style={
                    'color':'white',
                    'font-size':'16px',
                    'font-weight':'bold',
                    'textAlign':'left',
                    'float':'left'
                }),

            html.Div(dcc.Link('About Us', href='/about-us',
                style={
                    'color':'white',
                    'font-size': '16px',
                    'font-weight':'bold',
                    'textAlign':'right',
                    'float':'right',
                    'margin-right':'15px'
                })),
    
            html.Div(dcc.Link('Home', href='/',
                style={
                    'color':'white',
                    'font-size': '16px',
                    'font-weight':'bold',
                    'textAlign':'right',
                    'float':'right',
                    'margin-right':'15px'
                }))

        ],
            style={
                'background-color':'#003262',
                'height': '25px',
                'padding':'10px',
                'margin': '-10px -10px 20px -10px'
            })

# add callback for toggling the collapse on small screens



footer = html.Footer([
    html.Div(children=["Made with ❤️ in Berkeley, CA."], style={
        
        'color': 'white',
        'font-size': '16px'
        
    })],style={
            'textAlign':'left',
            'background-color': '#333',
            'overflow': 'hidden',
            'position': 'relative',
            'bottom': '0',
            'width': '100%',
            'height': '25px',
            'padding': '10px',
            'margin': '20px -10px -10px -10px'
           })

fig = go.Figure()
app.layout = html.Div(children=[
    
    navbar,
    
    html.Div(id='display'),
    
    footer


    ])


@app.callback(dash.dependencies.Output('display', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display(pathname):
    if pathname == '/about-us':
        return html.Iframe(srcDoc = str(open('about.html', 'r').read()), width = 1500, height = 1050)
    
    if pathname == "/":
        return html.Div([html.Div(children=[
            
              html.H1(children=["Welcome to ApartmentRanker!"],
           style={
               'textAlign':'center'
           }),
            
        html.H3(children="Tell us about yourself!",
               style={
                   'textAlign':'center'
               }),
        
        html.Div([
            dcc.Markdown(children=
                        """
                        ApartmentRanker works best if you answer each question honestly as we use this data to match suitable apartments with you. Simply fill out the required questions and see apartments around you today!
                        """)
        ]),
        
        html.Div(children=[
            
            html.H5(children="First, let's talk requirements.",
                    style={'textAlign':'center'}),
            
            html.Div(children=[
                html.Label('Number of bedrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bedrooms-state',
                    options=[
                        {'label': '0 (studio)','value':0.0},
                        {'label': '1 bedroom','value':1.0},
                        {'label': '2 bedrooms','value':2.0},
                        {'label': '3 bedrooms','value':3.0},
                        {'label': '4 bedrooms','value':4.0},
                        {'label': '5 bedrooms','value':5.0}
                    ]),

                html.Label('Number of bathrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bathrooms-state',
                    options=[
                        {'label': '1 bathroom','value':1.0},
                        {'label': '2 bathrooms','value':2.0},
                        {'label': '3 bathrooms','value':3.0},
                        {'label': '4 bathrooms','value':4.0},
                        {'label': '5 bathrooms','value':5.0}
                    ]),

                ],style={'columnCount':2}),
        
            html.Label('Maximum monthly rental price',style={"font-weight":"bold"}),
            dcc.Slider(
                id='price_max-state',
                min=1000,
                max=10000,
                marks={i: '${}'.format(i) for i in range (1000,11000,1000)}
            ),
            
            html.Label('Preferred neighborhood',style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='neighborhood-state',
                options=neighborhood_options,
                multi=False)
        
        ]),
        
        html.Div(children=[
            
            html.H5(children="One last thing, tell us about your priorities!",
                    style={'textAlign':'center'}),
            
            html.Label("1st priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority1-state',
                value='grocer',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("2nd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority2-state',
                value='nightlife',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),    
                 
            html.Label("3rd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority3-state',
                value='noise',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),   
                 
            html.Label("4th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority4-state',
                value='food',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("5th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority5-state',
                value='subway',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),  
                 
            html.Label("6th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority6-state',
                value='trees',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),                 
            ],
                style=
                    {
                        'margin-top':'10px',
                        'margin-bottom':'10px'
                    }),
        
        html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
        
        # INITIAL RECOMMENDATION 
            
        html.Div(id='hide-me-success',children=[
            
            html.H4(children="Woot! 🎉", style={'textAlign':'left','margin-bottom':'-3px','font-weight':'bold'}),

            html.Div(children="We found a recommendation for you.",style={'textAlign':'left'}),
            
            html.Div(id='mappy-boi',
                    style={
                        'width':'325px',
                        'height':'325px',
                        'float':'left',
                        'margin-right':'20px',
                        'margin-top':'10px'
                    }),
            
            html.Div(children=[
                html.H5(id='initial_recc_name', style={'margin-bottom':'-10px', 'font-weight':'bold'}),
                html.H6(id='initial_recc_neigh', style={'margin-bottom':'-2px', 'font-style':'oblique'}),
                html.Div(id='initial_recc_sent')],
                    style={
                        'textAlign':'left'
                    }),
            html.Div(children=[
                html.Li(id='grocer-num'),
                html.Li(id='nightlife-num'),
                html.Li(id='noise-num'),
                html.Li(id='resto-num'),
                html.Li(id='sub-num'),
                html.Li(id='tree-num')],
                    style={
                        'margin-top': '10px',
                        'margin-left':'50px',
                        'font-size':'12px',
                        'textAlign':'left'
                    }),
            
            html.Div(children=[
                html.Button(id='like-recommendation',children="👍"),
                html.Button(id='dislike-recommendation',children="👎")],
                    style={
                       "columnCount":2 
                    })
            ],
                style={
                    'display':'none',
                    'margin-bottom':'20px'
                }),

        html.Div(id='hide-me-failure', children=[
            
            html.H4(children="Hmm...🤔", style={'textAlign':'left','margin-bottom':'-3px','font-weight':'bold'}),
            html.Div(children="We couldn't find anything with this criteria.",style={'textAlign':'left'})],
                style={
                    'display':'none'
                }),
        
        # SIMILAR LISTINGS USING RECOMMENDATION ENGINE
        
        html.Div(id='hide-me-secondary', children=[
        
            html.H4(children="Here are more listings you might enjoy! 💡",
                   style={
                       'textAlign':'left',
                       'margin-bottom':'-3px',
                       'font-weight':'bold'
                   }),

            html.Div(children="These listings were suggested to you using our content-based recommendation engine based off our initial recommendation to you.",
                    style={
                        'textAlign':'left',
                        'font-style':'oblique',
                        'line-height':'120%'
                    }),
            
            html.Div(id='similar-1', children=[
            
                html.Div(id='similar-1-map',
                        style={
                            'width':'325px',
                            'height':'325px',
                            'float':'left',
                            'margin-right':'20px',
                            'margin-top':'10px',
                            'background':"#B9D3B6"
                        }),
                
                html.Div(children=[
                
                    html.Div(children=[
                        html.H5(id='Similar Apartment 1 name', style={'margin-bottom':'-10px', 'font-weight':'bold'}),
                        html.H6(id='Similar Apartment 1 neigh', style={'margin-bottom':'-2px', 'font-style':'oblique'}),
                        html.Div(id='Similar Apartment 1 desc')],
                            style={
                                'textAlign':'left'
                            }),

                    html.Div(children=[
                        html.Li(id='grocer-num-1'),
                        html.Li(id='nightlife-num-1'),
                        html.Li(id='noise-num-1'),
                        html.Li(id='resto-num-1'),
                        html.Li(id='sub-num-1'),
                        html.Li(id='tree-num-1')],
                            style={
                                'margin-top': '10px',
                                'margin-left':'50px',
                                'font-size':'12px',
                                'textAlign':'left'
                            })],style={'padding':'5px 0px 5px 0px'}
                    )],style={'height':'325px','margin':'10px 0px 10px 0px'}),
            
            html.Div(id='similar-2', children=[
            
                html.Div(id='similar-2-map',
                        style={
                            'width':'325px',
                            'height':'325px',
                            'float':'left',
                            'margin-right':'20px',
                            'margin-top':'10px',
                            'background':"#B9D3B6"
                        }),
                
                html.Div(children=[
                
                    html.Div(children=[
                        html.H5(id='Similar Apartment 2 name', style={'margin-bottom':'-10px', 'font-weight':'bold'}),
                        html.H6(id='Similar Apartment 2 neigh', style={'margin-bottom':'-2px', 'font-style':'oblique'}),
                        html.Div(id='Similar Apartment 2 desc')],
                            style={
                                'textAlign':'left'
                            }),

                    html.Div(children=[
                        html.Li(id='grocer-num-2'),
                        html.Li(id='nightlife-num-2'),
                        html.Li(id='noise-num-2'),
                        html.Li(id='resto-num-2'),
                        html.Li(id='sub-num-2'),
                        html.Li(id='tree-num-2')],
                            style={
                                'margin-top': '10px',
                                'margin-left':'50px',
                                'font-size':'12px',
                                'textAlign':'left'
                            })],style={'padding':'5px 0px 5px 0px'}
                    )],style={'height':'325px','margin':'10px 0px 10px 0px'}),        
            
            html.Div(id='similar-3', children=[
            
                html.Div(id='similar-3-map',
                        style={
                            'width':'325px',
                            'height':'325px',
                            'float':'left',
                            'margin-right':'20px',
                            'margin-top':'10px',
                            'background':"#B9D3B6"
                        }),
                
                html.Div(children=[
                
                    html.Div(children=[
                        html.H5(id='Similar Apartment 3 name', style={'margin-bottom':'-10px', 'font-weight':'bold'}),
                        html.H6(id='Similar Apartment 3 neigh', style={'margin-bottom':'-2px', 'font-style':'oblique'}),
                        html.Div(id='Similar Apartment 3 desc')],
                            style={
                                'textAlign':'left'
                            }),

                    html.Div(children=[
                        html.Li(id='grocer-num-3'),
                        html.Li(id='nightlife-num-3'),
                        html.Li(id='noise-num-3'),
                        html.Li(id='resto-num-3'),
                        html.Li(id='sub-num-3'),
                        html.Li(id='tree-num-3')],
                            style={
                                'margin-top': '10px',
                                'margin-left':'50px',
                                'font-size':'12px',
                                'textAlign':'left'
                            })],style={'padding':'5px 0px 5px 0px'}
                    )],style={'height':'325px','margin':'10px 0px 10px 0px'})
            
        ],
                style={
                    'display':'none'
                }),
            
    ], # <----- Ends the ENTIRE div for app_layout
        style=
            {"margin":"0 auto",
             'max-width':"650px",
             'textAlign':'center'
            }
    )])
    
@app.callback(
    [Output('mappy-boi','children'),
     Output('initial_recc_name','children'),
     Output('initial_recc_neigh','children'),
     Output('initial_recc_sent','children'),
     Output('grocer-num','children'),
     Output('nightlife-num','children'),
     Output('noise-num','children'),
     Output('resto-num','children'),
     Output('sub-num','children'),
     Output('tree-num','children'),
     Output('hide-me-success','style'),
     Output('hide-me-failure','style')],
    [Input('submit-button-state','n_clicks')],
    [State('bedrooms-state','value'),
    State('bathrooms-state','value'), 
    State('price_max-state','value'),
    State('neighborhood-state','value'),
    State('priority1-state','value'),
    State('priority2-state','value'),
    State('priority3-state','value'),
    State('priority4-state','value'),
    State('priority5-state','value'),
    State('priority6-state','value')])
def initial_rec(n_clicks,beds,bath,price,neighbor,p1,p2,p3,p4,p5,p6):
    #preferences is a list with the ranked features of
    # grocers_normalized
    # nightlife_normalized
    # noise_normalized
    # restaurants_normalized
    # subways_normalized
    # trees_normalized
    
    global new_df
    global prev_recc

    if n_clicks < 1:
        return
    
    counter=1

    tree_pref = 0
    subway_pref = 0
    grocer_pref = 0
    noise_pref = 0
    restaurants_pref = 0
    nightlife_pref = 0

    for p in [p1,p2,p3,p4,p5,p6]:
        if p=="trees":
            tree_pref=counter
            counter+=1
            continue
        if p=="subway":
            subway_pref=counter
            counter+=1
            continue
        if p=="grocer":
            grocer_pref=counter
            counter+=1
            continue
        if p=="noise":
            noise_pref=counter
            counter+=1
            continue
        if p=="food":
            restaurants_pref=counter
            counter+=1
            continue
        if p=="nightlife":
            nightlife_pref=counter
            counter+=1
            continue
            
    preferences = np.array([grocer_pref, nightlife_pref, noise_pref, restaurants_pref, subway_pref, tree_pref])
    
    new_df = full_df[(full_df['Bedrooms'] == beds) & (full_df['Bathrooms'] == bath) & (full_df['Monthly Rental Price'] <= price) & (full_df['Neighborhood'] == neighbor)]
    new_df["User Score"] = 0.0
    
    scores = []
    for i in range(new_df.shape[0]):
        score = preferences * np.array(new_df.iloc[i, 19:25] + 10)
        scores.append(sum(score))
    new_df['User Score'] = scores
    new_df = new_df.sort_values(by = ['User Score'], ascending = False)
    
    counter_1 = 0
#     print(new_df.head())
    try:
        while True:
            potential = new_df.iloc[counter_1, :]
            if potential not in dislikes:
                break
            else:
                counter_1 = counter_1 + 1
        result = potential
        first_recommendation = potential

        #recc = "We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) + " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms."
        apt_name = result['Location']
        apt_neigh = result['Neighborhood']
        recc_details = "This listing is $" + str(int(result['Monthly Rental Price'])) + " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms."
        apt_grocers = str(result['num_grocers']) + " grocery stores nearby." 
        apt_nightlife = str(result['num_nightlife']) + " nightlife spots nearby."
        apt_noise = str(result['num_noise'])  + " noise complaints in the last three months."
        apt_restos = str(result['num_restaurants']) + " restaurants or cafes nearby."
        apt_transit = str(result['num_subways'])  + " subway stops in walking distance."
        apt_trees = str(result['num_trees']) + " trees on nearby streets."
        
        fig = go.Figure(go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
        )
    )

        fig.update_layout(
        hovermode="closest",
        mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=first_recommendation["Latitude"],
            lon=first_recommendation["Longitude"]
        ),
        pitch=0,
        zoom=15
        )
    )

        trace_1 = go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
    )


        layout = go.Layout(
        hovermode="closest",
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=first_recommendation["Latitude"],
                lon=first_recommendation["Longitude"]
            ),
            pitch=0,
            zoom=15
        ),
        margin=dict(l=0,r=0,t=10,b=10),
        width=325,
        height=325
    )


        fig = go.Figure(data = [trace_1], layout = layout)

        htmlPlot = dcc.Graph(id = 'plot', figure = fig)
        prev_recc = result['Listing ID']
        return htmlPlot, apt_name, apt_neigh, recc_details, apt_grocers, apt_nightlife, apt_noise, apt_restos, apt_transit, apt_trees, {'display':'block'}, {'display':'none'}
    except:
        return None, "None", "None", "None", "None", "None", "None", "None", "None", "There is currently no listing with the given criteria.", {'display':'none'}, {'display':'block'}

@app.callback(
    [Output('similar-1-map','children'),
     Output('Similar Apartment 1 name','children'),
     Output('Similar Apartment 1 neigh','children'),
     Output('Similar Apartment 1 desc','children'),
     Output('grocer-num-1','children'),
     Output('nightlife-num-1','children'),
     Output('noise-num-1','children'),
     Output('resto-num-1','children'),
     Output('sub-num-1','children'),
     Output('tree-num-1','children'),
     Output('similar-2-map','children'),
     Output('Similar Apartment 2 name','children'),
     Output('Similar Apartment 2 neigh','children'),
     Output('Similar Apartment 2 desc','children'),
     Output('grocer-num-2','children'),
     Output('nightlife-num-2','children'),
     Output('noise-num-2','children'),
     Output('resto-num-2','children'),
     Output('sub-num-2','children'),
     Output('tree-num-2','children'),
     Output('similar-3-map','children'),
     Output('Similar Apartment 3 name','children'),
     Output('Similar Apartment 3 neigh','children'),
     Output('Similar Apartment 3 desc','children'),
     Output('grocer-num-3','children'),
     Output('nightlife-num-3','children'),
     Output('noise-num-3','children'),
     Output('resto-num-3','children'),
     Output('sub-num-3','children'),
     Output('tree-num-3','children'),
     Output('hide-me-secondary','style')],
    [Input('like-recommendation','n_clicks')])
def content_recommender(recc_id):
    global new_df
    
    new_df = new_df[~new_df['Listing ID'].isin(dislikes)]
    vals = new_df.loc[:,["grocers_normalized", "nightlife_normalized", "restaurants_normalized", "subways_normalized", "trees_normalized"]] + 10
    cosine_sim = cosine_similarity(vals)
    indices = pd.Series(new_df['Listing ID']).reset_index()
    
    address = prev_recc

    recommended_addresses = []
    idx = indices[indices == address].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:4].index)
    for i in top_10_indexes:
            recommended_addresses.append(list(vals.index)[i])
            
    #SIMILAR 1 DATA
    sim_a = new_df.loc[recommended_addresses[0]]
    sim_a_name = sim_a['Location']
    sim_a_neigh = sim_a['Neighborhood']
    sim_a_disc = "This listing is $" + str(int(sim_a['Monthly Rental Price'])) + " per month. It has " + str(int(sim_a['Bedrooms'])) + " bedrooms and " + str(int(sim_a['Bathrooms'])) + " bathrooms."
    sim_a_grocer = str(sim_a['num_grocers']) + " grocery stores nearby." 
    sim_a_night = str(sim_a['num_nightlife']) + " nightlife spots nearby."
    sim_a_noise = str(sim_a['num_noise']) + " noise complaints in the last three months."
    sim_a_resto = str(sim_a['num_restaurants']) + " restaurants or cafes nearby."
    sim_a_sub = str(sim_a['num_subways']) + " subway stops in walking distance."
    sim_a_tree = str(sim_a['num_trees']) + " trees on nearby streets."
    
    
    #SIMILAR 2 DATA
    sim_b = new_df.loc[recommended_addresses[1]]
    sim_b_name = sim_b['Location']
    sim_b_neigh = sim_b['Neighborhood']
    sim_b_disc = "This listing is $" + str(int(sim_b['Monthly Rental Price'])) + " per month. It has " + str(int(sim_b['Bedrooms'])) + " bedrooms and " + str(int(sim_b['Bathrooms'])) + " bathrooms."
    sim_b_grocer = str(sim_b['num_grocers']) + " grocery stores nearby."
    sim_b_night = str(sim_b['num_nightlife']) + " nightlife spots nearby."
    sim_b_noise = str(sim_b['num_noise']) + " noise complaints in the last three months."
    sim_b_resto = str(sim_b['num_restaurants']) + " restaurants or cafes nearby."
    sim_b_sub = str(sim_b['num_subways']) + " subway stops in walking distance."
    sim_b_tree = str(sim_b['num_trees']) + " trees on nearby streets."
        
    #SIMILAR 3 DATA
    sim_c = new_df.loc[recommended_addresses[2]]
    sim_c_name = sim_c['Location']
    sim_c_neigh = sim_c['Neighborhood']
    sim_c_disc = "This listing is $" + str(int(sim_c['Monthly Rental Price'])) + " per month. It has " + str(int(sim_c['Bedrooms'])) + " bedrooms and " + str(int(sim_c['Bathrooms'])) + " bathrooms."
    sim_c_grocer = str(sim_c['num_grocers']) + " grocery stores nearby."
    sim_c_night = str(sim_c['num_nightlife']) + " nightlife spots nearby."
    sim_c_noise = str(sim_c['num_noise']) + " noise complaints in the last three months."
    sim_c_resto = str(sim_c['num_restaurants']) + " restaurants or cafes nearby."
    sim_c_sub = str(sim_c['num_subways']) + " subway stops in walking distance."
    sim_c_tree = str(sim_c['num_trees']) + " trees on nearby streets."
        
    # VIZ A

    fig1 = go.Figure(go.Scattermapbox(
            lat=[new_df.loc[recommended_addresses[0]]['Latitude']],
            lon=[new_df.loc[recommended_addresses[0]]['Longitude']],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=14
            ),
            text=[new_df.loc[recommended_addresses[0]]['Location']],
        ))

    fig1.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=new_df.loc[recommended_addresses[0]]['Latitude'],
                lon=new_df.loc[recommended_addresses[0]]['Longitude']
            ),
            pitch=0,
            zoom=15),
        margin=dict(l=0,r=0,t=10,b=10),
        width=325,
        height=325
    )

    htmlPlot1 = dcc.Graph(id='similar-1-map', figure=fig1)

    # VIZ B

    fig2 = go.Figure(go.Scattermapbox(
            lat=[new_df.loc[recommended_addresses[1]]['Latitude']],
            lon=[new_df.loc[recommended_addresses[1]]['Longitude']],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=14
            ),
            text=[new_df.loc[recommended_addresses[1]]['Location']],
        ))

    fig2.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=new_df.loc[recommended_addresses[1]]['Latitude'],
                lon=new_df.loc[recommended_addresses[1]]['Longitude']
            ),
            pitch=0,
            zoom=15),
        margin=dict(l=0,r=0,t=10,b=10),
        width=325,
        height=325
    )

    htmlPlot2 = dcc.Graph(id='similar-2-map', figure=fig2)

    # VIZ 3

    fig3 = go.Figure(go.Scattermapbox(
            lat=[new_df.loc[recommended_addresses[2]]['Latitude']],
            lon=[new_df.loc[recommended_addresses[2]]['Longitude']],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=14
            ),
            text=[new_df.loc[recommended_addresses[2]]['Location']],
        ))

    fig3.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=new_df.loc[recommended_addresses[2]]['Latitude'],
                lon=new_df.loc[recommended_addresses[2]]['Longitude']
            ),
            pitch=0,
            zoom=15),
        margin=dict(l=0,r=0,t=10,b=10),
        width=325,
        height=325
    )

    htmlPlot3 = dcc.Graph(id='similar-3-map', figure=fig3)
    
    return htmlPlot1, sim_a_name, sim_a_neigh, sim_a_disc, sim_a_grocer, sim_a_night, sim_a_noise, sim_a_resto, sim_a_sub, sim_a_tree, htmlPlot2, sim_b_name, sim_b_neigh, sim_b_disc, sim_b_grocer, sim_b_night, sim_b_noise, sim_b_resto, sim_b_sub, sim_b_tree, htmlPlot3, sim_c_name, sim_c_neigh, sim_c_disc, sim_c_grocer, sim_c_night, sim_c_noise, sim_c_resto, sim_c_sub, sim_c_tree, {'display':'block'}
        
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2020 01:23:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 01:23:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 01:23:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 01:23:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 01:23:21] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "//anaconda3/lib/python3.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, output

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "//anaconda3/lib/python3.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, output

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "//anaconda3/lib/python3.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, output

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "//anaconda3/lib/python3.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, output

127.0.0.1 - - [04/May/2020 01:23:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/May/2020 01:23:23] "POST /_dash-update-component HTTP/1.1" 200 -
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:535: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:541: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

127.0.0.1 - - [04/May/2020 01:23:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 01:23:38] "POST /_dash-update-component HTTP

In [74]:
        fig = go.Figure(go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
        )
    )

        fig.update_layout(
        hovermode="closest",
        mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=first_recommendation["Latitude"],
            lon=first_recommendation["Longitude"]
        ),
        pitch=0,
        zoom=15
        )
    )

        trace_1 = go.Scattermapbox(
        lat=pd.Series(first_recommendation["Latitude"]),
        lon=pd.Series(first_recommendation["Longitude"]),
        mode="markers",
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=first_recommendation["Location"],
    )


        layout = go.Layout(
        hovermode="closest",
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=first_recommendation["Latitude"],
                lon=first_recommendation["Longitude"]
            ),
            pitch=0,
            zoom=15
        ),
        margin=dict(l=0,r=0,t=10,b=10),
        width=325,
        height=325
    )


        fig = go.Figure(data = [trace_1], layout = layout)

        htmlPlot = dcc.Graph(id = 'plot', figure = fig)

In [100]:
# VIZ A

fig1 = go.Figure(go.Scattermapbox(
        lat=[new_df.loc[recommended_addresses[0]]['Latitude']],
        lon=[new_df.loc[recommended_addresses[0]]['Longitude']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=[new_df.loc[recommended_addresses[0]]['Location']],
    ))

fig1.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=new_df.loc[recommended_addresses[0]]['Latitude'],
            lon=new_df.loc[recommended_addresses[0]]['Longitude']
        ),
        pitch=0,
        zoom=15),
    margin=dict(l=0,r=0,t=10,b=10),
    width=325,
    height=325
)

htmlPlot1 = dcc.Graph(id='similar-1-map', figure=fig1)

# VIZ B

fig2 = go.Figure(go.Scattermapbox(
        lat=[new_df.loc[recommended_addresses[1]]['Latitude']],
        lon=[new_df.loc[recommended_addresses[1]]['Longitude']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=[new_df.loc[recommended_addresses[1]]['Location']],
    ))

fig2.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=new_df.loc[recommended_addresses[1]]['Latitude'],
            lon=new_df.loc[recommended_addresses[1]]['Longitude']
        ),
        pitch=0,
        zoom=15),
    margin=dict(l=0,r=0,t=10,b=10),
    width=325,
    height=325
)

htmlPlot2 = dcc.Graph(id='similar-2-map', figure=fig2)

# VIZ 3

fig3 = go.Figure(go.Scattermapbox(
        lat=[new_df.loc[recommended_addresses[2]]['Latitude']],
        lon=[new_df.loc[recommended_addresses[2]]['Longitude']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=[new_df.loc[recommended_addresses[2]]['Location']],
    ))

fig3.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=new_df.loc[recommended_addresses[2]]['Latitude'],
            lon=new_df.loc[recommended_addresses[2]]['Longitude']
        ),
        pitch=0,
        zoom=15),
    margin=dict(l=0,r=0,t=10,b=10),
    width=325,
    height=325
)

htmlPlot3 = dcc.Graph(id='similar-3-map', figure=fig3)